In [ ]:
import requests
import urllib.parse

def get_lat_long(address):
    # Append " Hong Kong" to the address for better geocoding
    full_address = address + " Hong Kong"
    # URL encode the address to handle spaces and special characters
    encoded_address = urllib.parse.quote(full_address)
    url = f"https://geodata.gov.hk/gs/api/v1.0.0/locationSearch?q={encoded_address}"

    try:
        # Add a timeout to the request
        response = requests.get(url, timeout=10)
        print(f"Request URL: {response.url}")
        print(f"Status Code: {response.status_code}")
        # Print response text only if debugging is needed or if status is not 200
        # print(f"Response Text: {response.text}")

        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        data = response.json()

        # Check if 'results' key exists, is a non-empty list, and the first item is a dict
        if isinstance(data, list) and data: # Check if data itself is the list
             first_result = data[0]
             if isinstance(first_result, dict):
                 lat = first_result.get("y")
                 lon = first_result.get("x")
                 if lat is not None and lon is not None:
                     return lat, lon
                 else:
                     print("Error: Latitude ('y') or Longitude ('x') not found in the first result.")
             else:
                 print(f"Error: First result is not a dictionary, but type {type(first_result)}. Content: {first_result}")
        # Handle cases where the API might return a dict containing results (less common based on observation but safer)
        elif isinstance(data, dict) and data.get("results") and isinstance(data["results"], list) and data["results"]:
             first_result = data["results"][0]
             if isinstance(first_result, dict):
                 lat = first_result.get("y")
                 lon = first_result.get("x")
                 if lat is not None and lon is not None:
                     return lat, lon
                 else:
                     print("Error: Latitude ('y') or Longitude ('x') not found in the first result.")
             else:
                 print(f"Error: First result in 'results' list is not a dictionary, but type {type(first_result)}. Content: {first_result}")
        else:
            print(f"Error: No valid results found in the response. Data received: {data}")

    except requests.exceptions.Timeout:
        print(f"Error: Request timed out for address: {full_address}")
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Status Code: {response.status_code}")
        print(f"Response Text: {response.text}") # Print response text on HTTP error
    except requests.exceptions.RequestException as req_err:
        print(f"Error during requests to {url}: {req_err}")
    except requests.exceptions.JSONDecodeError:
        print(f"Error: Failed to decode JSON response. Response text: {response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    # Raise ValueError if coordinates couldn't be obtained for any reason
    raise ValueError(f"Could not find location for address: {full_address}")

def main():
    address = input("Enter the address: ")
    if not address.strip():
        print("Address cannot be empty.")
        return
    try:
        lat, lon = get_lat_long(address)
        print(f"Latitude: {lat}, Longitude: {lon}")
    except ValueError as e:
        print(e)

if __name__ == "__main__":
    main()

https://geodata.gov.hk/gs/api/v1.0.0/locationSearch?q=swire%20hall%20Hong%20Kong
200
[{"addressZH":"香港大學太古堂","nameZH":"已登記的WiFi 熱點: 香港大學 (APID 07918)","districtZH":"中西區","x":832422.3,"y":816097.1,"nameEN":"Registered WiFi Access Points: The University of Hong Kong (APID 07918)","addressEN":"Swire Hall, Swire Building, The University of Hong Kong, Pokfulam Road","districtEN":"Central & Western"}]
Error: 'results' key not found, not a list, or is empty in the response. Data: [{'addressZH': '香港大學太古堂', 'nameZH': '已登記的WiFi 熱點: 香港大學 (APID 07918)', 'districtZH': '中西區', 'x': 832422.3, 'y': 816097.1, 'nameEN': 'Registered WiFi Access Points: The University of Hong Kong (APID 07918)', 'addressEN': 'Swire Hall, Swire Building, The University of Hong Kong, Pokfulam Road', 'districtEN': 'Central & Western'}]
Could not find location for address: swire hall Hong Kong
